In [1]:
# For tips on running notebooks in Google Colab, see
# https://pytorch.org/tutorials/beginner/colab
%matplotlib inline

Neural Transfer Using PyTorch
=============================

**Author**: [Alexis Jacq](https://alexis-jacq.github.io)

**Edited by**: [Winston Herring](https://github.com/winston6)

Introduction
------------

This tutorial explains how to implement the [Neural-Style
algorithm](https://arxiv.org/abs/1508.06576) developed by Leon A. Gatys,
Alexander S. Ecker and Matthias Bethge. Neural-Style, or
Neural-Transfer, allows you to take an image and reproduce it with a new
artistic style. The algorithm takes three images, an input image, a
content-image, and a style-image, and changes the input to resemble the
content of the content-image and the artistic style of the style-image.

![](https://pytorch.org/tutorials/_static/img/neural-style/neuralstyle.png)


Underlying Principle
====================

The principle is simple: we define two distances, one for the content
($D_C$) and one for the style ($D_S$). $D_C$ measures how different the
content is between two images while $D_S$ measures how different the
style is between two images. Then, we take a third image, the input, and
transform it to minimize both its content-distance with the
content-image and its style-distance with the style-image. Now we can
import the necessary packages and begin the neural transfer.

Importing Packages and Selecting a Device
=========================================

Below is a list of the packages needed to implement the neural transfer.

-   `torch`, `torch.nn`, `numpy` (indispensables packages for neural
    networks with PyTorch)
-   `torch.optim` (efficient gradient descents)
-   `PIL`, `PIL.Image`, `matplotlib.pyplot` (load and display images)
-   `torchvision.transforms` (transform PIL images into tensors)
-   `torchvision.models` (train or load pretrained models)
-   `copy` (to deep copy the models; system package)


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from PIL import Image
import matplotlib.pyplot as plt

import torchvision.transforms as transforms
from torchvision.models import vgg19, VGG19_Weights

import copy

Next, we need to choose which device to run the network on and import
the content and style images. Running the neural transfer algorithm on
large images takes longer and will go much faster when running on a GPU.
We can use `torch.cuda.is_available()` to detect if there is a GPU
available. Next, we set the `torch.device` for use throughout the
tutorial. Also the `.to(device)` method is used to move tensors or
modules to a desired device.


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.set_default_device(device)

Loss Functions
==============

Content Loss
------------

The content loss is a function that represents a weighted version of the
content distance for an individual layer. The function takes the feature
maps $F_{XL}$ of a layer $L$ in a network processing input $X$ and
returns the weighted content distance $w_{CL}.D_C^L(X,C)$ between the
image $X$ and the content image $C$. The feature maps of the content
image($F_{CL}$) must be known by the function in order to calculate the
content distance. We implement this function as a torch module with a
constructor that takes $F_{CL}$ as an input. The distance
$\|F_{XL} - F_{CL}\|^2$ is the mean square error between the two sets of
feature maps, and can be computed using `nn.MSELoss`.

We will add this content loss module directly after the convolution
layer(s) that are being used to compute the content distance. This way
each time the network is fed an input image the content losses will be
computed at the desired layers and because of auto grad, all the
gradients will be computed. Now, in order to make the content loss layer
transparent we must define a `forward` method that computes the content
loss and then returns the layer's input. The computed loss is saved as a
parameter of the module.


In [4]:
class ContentLoss(nn.Module):

    def __init__(self, target,):
        super(ContentLoss, self).__init__()
        # we 'detach' the target content from the tree used
        # to dynamically compute the gradient: this is a stated value,
        # not a variable. Otherwise the forward method of the criterion
        # will throw an error.
        self.target = target.detach()

    def forward(self, input):
        self.loss = F.mse_loss(input, self.target)
        return input

<div style="background-color: #54c7ec; color: #fff; font-weight: 700; padding-left: 10px; padding-top: 5px; padding-bottom: 5px"><strong>NOTE:</strong></div>

<div style="background-color: #f3f4f7; padding-left: 10px; padding-top: 10px; padding-bottom: 10px; padding-right: 10px">

<p>: although this module is named <code>ContentLoss</code>, itis not a true PyTorch Loss function. If you want to define your contentloss as a PyTorch Loss function, you have to create a PyTorch autograd functionto recompute/implement the gradient manually in the <code>backward</code>method.</p>

</div>



Style Loss
==========

The style loss module is implemented similarly to the content loss
module. It will act as a transparent layer in a network that computes
the style loss of that layer. In order to calculate the style loss, we
need to compute the gram matrix $G_{XL}$. A gram matrix is the result of
multiplying a given matrix by its transposed matrix. In this application
the given matrix is a reshaped version of the feature maps $F_{XL}$ of a
layer $L$. $F_{XL}$ is reshaped to form $\hat{F}_{XL}$, a $K$x$N$
matrix, where $K$ is the number of feature maps at layer $L$ and $N$ is
the length of any vectorized feature map $F_{XL}^k$. For example, the
first line of $\hat{F}_{XL}$ corresponds to the first vectorized feature
map $F_{XL}^1$.

Finally, the gram matrix must be normalized by dividing each element by
the total number of elements in the matrix. This normalization is to
counteract the fact that $\hat{F}_{XL}$ matrices with a large $N$
dimension yield larger values in the Gram matrix. These larger values
will cause the first layers (before pooling layers) to have a larger
impact during the gradient descent. Style features tend to be in the
deeper layers of the network so this normalization step is crucial.


In [5]:
def gram_matrix(input):
    a, b, c, d = input.size()  # a=batch size(=1)
    # b=number of feature maps
    # (c,d)=dimensions of a f. map (N=c*d)

    features = input.view(a * b, c * d)  # resize F_XL into \hat F_XL

    G = torch.mm(features, features.t())  # compute the gram product

    # we 'normalize' the values of the gram matrix
    # by dividing by the number of element in each feature maps.
    return G.div(a * b * c * d)

Now the style loss module looks almost exactly like the content loss
module. The style distance is also computed using the mean square error
between $G_{XL}$ and $G_{SL}$.


In [6]:
class StyleLoss(nn.Module):

    def __init__(self, target_feature):
        super(StyleLoss, self).__init__()
        self.target = gram_matrix(target_feature).detach()

    def forward(self, input):
        G = gram_matrix(input)
        self.loss = F.mse_loss(G, self.target)
        return input

Importing the Model
===================

Now we need to import a pretrained neural network. We will use a 19
layer VGG network like the one used in the paper.

PyTorch's implementation of VGG is a module divided into two child
`Sequential` modules: `features` (containing convolution and pooling
layers), and `classifier` (containing fully connected layers). We will
use the `features` module because we need the output of the individual
convolution layers to measure content and style loss. Some layers have
different behavior during training than evaluation, so we must set the
network to evaluation mode using `.eval()`.


In [7]:
cnn = vgg19(weights=VGG19_Weights.DEFAULT).features.eval()

Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth
100%|██████████| 548M/548M [00:05<00:00, 103MB/s]


Additionally, VGG networks are trained on images with each channel
normalized by mean=\[0.485, 0.456, 0.406\] and std=\[0.229, 0.224,
0.225\]. We will use them to normalize the image before sending it into
the network.


In [8]:
cnn_normalization_mean = torch.tensor([0.485, 0.456, 0.406])
cnn_normalization_std = torch.tensor([0.229, 0.224, 0.225])

# create a module to normalize input image so we can easily put it in a
# ``nn.Sequential``
class Normalization(nn.Module):
    def __init__(self, mean, std):
        super(Normalization, self).__init__()
        # .view the mean and std to make them [C x 1 x 1] so that they can
        # directly work with image Tensor of shape [B x C x H x W].
        # B is batch size. C is number of channels. H is height and W is width.
        self.mean = torch.tensor(mean).view(-1, 1, 1)
        self.std = torch.tensor(std).view(-1, 1, 1)

    def forward(self, img):
        # normalize ``img``
        return (img - self.mean) / self.std

A `Sequential` module contains an ordered list of child modules. For
instance, `vgg19.features` contains a sequence (`Conv2d`, `ReLU`,
`MaxPool2d`, `Conv2d`, `ReLU`...) aligned in the right order of depth.
We need to add our content loss and style loss layers immediately after
the convolution layer they are detecting. To do this we must create a
new `Sequential` module that has content loss and style loss modules
correctly inserted.


In [9]:
# desired depth layers to compute style/content losses :
content_layers_default = ['conv_4']
style_layers_default = ['conv_1', 'conv_2', 'conv_3', 'conv_4', 'conv_5']

def get_style_model_and_losses(cnn, normalization_mean, normalization_std,
                               style_img, content_img,
                               content_layers=content_layers_default,
                               style_layers=style_layers_default):
    # normalization module
    normalization = Normalization(normalization_mean, normalization_std)

    # just in order to have an iterable access to or list of content/style
    # losses
    content_losses = []
    style_losses = []

    # assuming that ``cnn`` is a ``nn.Sequential``, so we make a new ``nn.Sequential``
    # to put in modules that are supposed to be activated sequentially
    model = nn.Sequential(normalization)

    i = 0  # increment every time we see a conv
    for layer in cnn.children():
        if isinstance(layer, nn.Conv2d):
            i += 1
            name = 'conv_{}'.format(i)
        elif isinstance(layer, nn.ReLU):
            name = 'relu_{}'.format(i)
            # The in-place version doesn't play very nicely with the ``ContentLoss``
            # and ``StyleLoss`` we insert below. So we replace with out-of-place
            # ones here.
            layer = nn.ReLU(inplace=False)
        elif isinstance(layer, nn.MaxPool2d):
            name = 'pool_{}'.format(i)
        elif isinstance(layer, nn.BatchNorm2d):
            name = 'bn_{}'.format(i)
        else:
            raise RuntimeError('Unrecognized layer: {}'.format(layer.__class__.__name__))

        model.add_module(name, layer)

        if name in content_layers:
            # add content loss:
            target = model(content_img).detach()
            content_loss = ContentLoss(target)
            model.add_module("content_loss_{}".format(i), content_loss)
            content_losses.append(content_loss)

        if name in style_layers:
            # add style loss:
            target_feature = model(style_img).detach()
            style_loss = StyleLoss(target_feature)
            model.add_module("style_loss_{}".format(i), style_loss)
            style_losses.append(style_loss)

    # now we trim off the layers after the last content and style losses
    for i in range(len(model) - 1, -1, -1):
        if isinstance(model[i], ContentLoss) or isinstance(model[i], StyleLoss):
            break

    model = model[:(i + 1)]

    return model, style_losses, content_losses

Next, we select the input image. You can use a copy of the content image
or white noise.


In [10]:
# input_img = content_img.clone()
# # if you want to use white noise by using the following code:
# #
# # .. code-block:: python
# #
# #    input_img = torch.randn(content_img.data.size())

# # add the original input image to the figure:
# plt.figure()
# imshow(input_img, title='Input Image')

Gradient Descent
================

As Leon Gatys, the author of the algorithm, suggested
[here](https://discuss.pytorch.org/t/pytorch-tutorial-for-neural-transfert-of-artistic-style/336/20?u=alexis-jacq),
we will use L-BFGS algorithm to run our gradient descent. Unlike
training a network, we want to train the input image in order to
minimize the content/style losses. We will create a PyTorch L-BFGS
optimizer `optim.LBFGS` and pass our image to it as the tensor to
optimize.


In [11]:
def get_input_optimizer(input_img):
    # this line to show that input is a parameter that requires a gradient
    optimizer = optim.LBFGS([input_img])
    return optimizer

Finally, we must define a function that performs the neural transfer.
For each iteration of the networks, it is fed an updated input and
computes new losses. We will run the `backward` methods of each loss
module to dynamically compute their gradients. The optimizer requires a
"closure" function, which reevaluates the module and returns the loss.

We still have one final constraint to address. The network may try to
optimize the input with values that exceed the 0 to 1 tensor range for
the image. We can address this by correcting the input values to be
between 0 to 1 each time the network is run.


In [12]:
def run_style_transfer(cnn, normalization_mean, normalization_std,
                       content_img, style_img, input_img, num_steps=300,
                       style_weight=1000000, content_weight=1):
    """Run the style transfer."""
    print('Building the style transfer model..')
    model, style_losses, content_losses = get_style_model_and_losses(cnn,
        normalization_mean, normalization_std, style_img, content_img)

    # We want to optimize the input and not the model parameters so we
    # update all the requires_grad fields accordingly
    input_img.requires_grad_(True)
    # We also put the model in evaluation mode, so that specific layers
    # such as dropout or batch normalization layers behave correctly.
    model.eval()
    model.requires_grad_(False)

    optimizer = get_input_optimizer(input_img)

    print('Optimizing..')
    run = [0]
    while run[0] <= num_steps:

        def closure():
            # correct the values of updated input image
            with torch.no_grad():
                input_img.clamp_(0, 1)

            optimizer.zero_grad()
            model(input_img)
            style_score = 0
            content_score = 0

            for sl in style_losses:
                style_score += sl.loss
            for cl in content_losses:
                content_score += cl.loss

            style_score *= style_weight
            content_score *= content_weight

            loss = style_score + content_score
            loss.backward()

            run[0] += 1
            if run[0] % num_steps == 0:
                print("run {}:".format(run))
                print('Style Loss : {:4f} Content Loss: {:4f}'.format(
                    style_score.item(), content_score.item()))
                print()

            return style_score + content_score

        optimizer.step(closure)

    # a last correction...
    with torch.no_grad():
        input_img.clamp_(0, 1)

    return input_img

Finally, we can run the algorithm.


In [13]:

# output = run_style_transfer(cnn, cnn_normalization_mean, cnn_normalization_std,
#                             content_img, style_img, input_img)

# plt.figure()
# imshow(output, title='Output Image')

# # sphinx_gallery_thumbnail_number = 4
# plt.ioff()
# plt.show()

In [14]:
from google.colab import drive
import os
import shutil

drive.mount('/content/drive')
dataset_path = '/content/drive/MyDrive/LoveDA/Train'
if (os.path.exists("./Train") == False):
    if (os.path.exists("/content/drive/MyDrive/LoveDA/Train.zip")):
        print("Dataset available on own drive, unzipping...")
        !unzip -q /content/drive/MyDrive/LoveDA/Train.zip -d ./
    else:
        print("Downloading dataset...")
        !wget -O Train.zip "https://zenodo.org/records/5706578/files/Train.zip?download=1"
        if(SAVE_ON_DRIVE):
            print("Saving dataset on drive...")
            !cp Train.zip /content/drive/MyDrive/LoveDA/
        !unzip -q Train.zip -d ./

else:
    print("Dataset already in local")

Mounted at /content/drive
Dataset available on own drive, unzipping...


# Dataset Definition

In [15]:
import os
import numpy as np
from PIL import Image
from torch.utils.data import Dataset
import torchvision.transforms as T
import random
import cv2


def pil_loader(path, color_type):
    # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
    with open(path, 'rb') as f:
        img = Image.open(f)
        return img.convert(color_type)

class LoveDADataset(Dataset):
    def __init__(self, baseTransform, augTransforms, split = 'Urban', typeDataset = 'Train', validation_ratio=0.2, seed=265637, filter = None):
        # Validate typeDataset input
        if typeDataset not in ['Train', 'Validation', 'Total']:
            raise ValueError("Invalid typeDataset. Expected 'Train' or 'Validation'.")
        self.directory = []
        directory_path = os.path.join('./Train', split, 'images_png')
        # Check if the directory exists
        if not os.path.exists(directory_path):
            raise FileNotFoundError(f"Directory not found: {directory_path}")
        # Get all image paths
        all_images = [os.path.join(directory_path, entry) for entry in os.listdir(directory_path) if os.path.isfile(os.path.join(directory_path, entry))]
        relative_all = [entry for entry in os.listdir(directory_path) if os.path.isfile(os.path.join(directory_path, entry))]
        if filter is not None:
            relative = [entry for entry in relative_all if entry in filter]
            all_images = [os.path.join(directory_path, entry) for entry in relative]

        # Split into training and validation sets
        if validation_ratio > 0.0:
          # Shuffle images for random splitting
          random.seed(seed)
          random.shuffle(all_images)
        # Split into training and validation sets
        split_idx = int(len(all_images) * (1 - validation_ratio))
        if typeDataset == 'Train':
            self.directory = all_images[:split_idx]
        elif typeDataset == 'Validation':
            self.directory = all_images[split_idx:]
        elif typeDataset == 'Total':
            self.directory = all_images
        else:
            raise ValueError("Invalid typeDataset. Expected 'Train' or 'Validation' or 'Total'.")
        self.baseTransforms = baseTransform
        self.augTransforms = augTransforms
        self.typeDataset = typeDataset
        # Print dataset size
        print(f"Dataset size: {len(self.directory)}")

    def __len__(self):
        return len(self.directory)

    def __getitem__(self, idx):
        image_path = self.directory[idx]
        image = pil_loader(image_path, 'RGB')

        base_transformed = self.baseTransforms(image=np.array(image))
        base_image = base_transformed['image']

        base_image = T.Compose([T.ToTensor()])(base_image)

        return base_image, image_path

# Filter rural images that have a lot of unclassified




In [16]:
from PIL import Image
import os
import torch
from tqdm import tqdm

def pil_loader(path, color_type):
    # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
    with open(path, 'rb') as f:
        img = Image.open(f)
        return img.convert(color_type)

urban_masks_path = "./Train/Urban/masks_png"
rural_masks_path = "./Train/Rural/masks_png"

In [17]:
import torch
import numpy as np

idx_to_class = {
    'Unclassified':0,
    'Background':1,
    'Building':2,
    'Road':3,
    'Water':4,
    'Barren':5,
    'Forest':6,
    'Agricultural':7
}


class_ids = [0, 1, 2, 3, 4, 5, 6, 7]

img_unlabel_count = dict()

total = 0
for masks_path in [rural_masks_path]:
    print(f"processing {masks_path}")

    loop = tqdm(os.listdir(masks_path))
    for filename in loop:
        loop.set_description(f"Processing {filename} from ") # update desc with filename

        mask = pil_loader(os.path.join(masks_path, filename), "L")

        # Convert the PIL Image to a PyTorch tensor
        mask_tensor = torch.from_numpy(np.array(mask))

        # Count the occurrences of each class using torch.unique
        unique_classes, counts = torch.unique(mask_tensor, return_counts=True)

        if idx_to_class['Unclassified'] in unique_classes:
          try:
            img_unlabel_count[filename] = counts[idx_to_class['Unclassified']]
          except:
            img_unlabel_count[filename] = 0

processing ./Train/Rural/masks_png


Processing 223.png from : 100%|██████████| 1366/1366 [00:25<00:00, 53.99it/s]


In [18]:

import operator

sorted_img_unlabel_count = dict(sorted(img_unlabel_count.items(), key=operator.itemgetter(1)))
print(sorted_img_unlabel_count)

# Calculate n percentile
import numpy as np
import operator

# Extract the counts (values) from the sorted dictionary
counts = list(sorted_img_unlabel_count.values())

k = 20
# Calculate the k-th percentile (median)
median = np.percentile(counts, k)
filtered_img_unlabel_count = {k: v for k, v in img_unlabel_count.items() if v < median}
filtered_rural = dict(sorted(filtered_img_unlabel_count.items(), key=operator.itemgetter(1)))
print(filtered_rural)


{'174.png': tensor(1), '947.png': tensor(64), '679.png': tensor(65), '203.png': tensor(66), '260.png': tensor(66), '1268.png': tensor(68), '897.png': tensor(69), '175.png': tensor(71), '118.png': tensor(72), '376.png': tensor(73), '793.png': tensor(76), '625.png': tensor(86), '1160.png': tensor(99), '233.png': tensor(110), '738.png': tensor(152), '1020.png': tensor(178), '257.png': tensor(189), '195.png': tensor(223), '715.png': tensor(263), '906.png': tensor(290), '1029.png': tensor(301), '946.png': tensor(364), '1095.png': tensor(426), '758.png': tensor(462), '438.png': tensor(515), '338.png': tensor(653), '495.png': tensor(704), '172.png': tensor(787), '361.png': tensor(1344), '844.png': tensor(8397), '523.png': tensor(12341), '1072.png': tensor(26359), '840.png': tensor(26534), '413.png': tensor(27545), '1136.png': tensor(27634), '1174.png': tensor(31168), '668.png': tensor(31657), '1339.png': tensor(32928), '952.png': tensor(33727), '91.png': tensor(36198), '398.png': tensor(40973

# Create loaders

In [19]:

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torch.nn.functional as F
from albumentations import Compose, HorizontalFlip, RandomRotate90, RandomScale, RandomCrop, GaussNoise, Rotate, Resize, OneOf, Normalize, ColorJitter, GaussianBlur
from albumentations.pytorch import ToTensorV2



#How big should be the image that we feed to the model?
RESIZE = 512
# DEFINE TRANSFORMATIONS HERE
# To Tensor is not needed since its performed inside the getitem


AUGMENTATIONS = {
    'None': Compose([
        Resize(RESIZE, RESIZE)
    ]),
}

transform = AUGMENTATIONS['None']

urban_dataset = LoveDADataset(baseTransform=transform, augTransforms=None ,split='Urban',
                                typeDataset='Train', validation_ratio=0.0)

urban_loader = DataLoader(urban_dataset, batch_size=1, shuffle=False,
                            num_workers=2, drop_last=True, pin_memory=True)

rural_dataset = LoveDADataset(baseTransform=transform, augTransforms=None ,split='Rural',
                                typeDataset='Train', validation_ratio=0.0, filter =filtered_rural.keys())

rural_loader = DataLoader(rural_dataset, batch_size=1, shuffle=True,
                            num_workers=2, drop_last=True, pin_memory=True,
                            generator=torch.Generator(device='cuda')) # Add this line





Dataset size: 1156
Dataset size: 39


/usr/local/lib/python3.11/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 2.0.2 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


# Style transfer create

In [ ]:
import os
import torch
import torchvision
import matplotlib.pyplot as plt
from tqdm import tqdm
#imshow


from PIL import Image
from torchvision import transforms, models

if not os.path.exists("./StyleTransfer/Urban/"):
  os.makedirs("./StyleTransfer/Urban/")
for urban, urban_path in tqdm(urban_loader):

  find_good_one = False
  while not find_good_one:
    try:
      rural, path = rural_loader.__iter__().__next__()
      print('Tried one')
      if path[0].split('/')[-1] in filtered_rural:
        find_good_one = True
    except StopIteration:
      print('StopIteration')
      rural_loader = DataLoader(rural_dataset, batch_size=1, shuffle=False,
                            num_workers=2, drop_last=True, pin_memory=True)

  urban_image = urban
  style_image = rural
  content_image = urban

  input_img = content_image.clone().detach()
  input_img.to('cuda')

  # plt.figure()
  # plt.title('Style Image')
  # plt.imshow(style_image[0].permute(1, 2, 0))

  # plt.figure()
  # plt.title('Content Image')
  # plt.imshow(content_image[0].permute(1,2,0))

  output = run_style_transfer(cnn.to('cuda'), cnn_normalization_mean.to('cuda'), cnn_normalization_std.to('cuda'),
                              content_image.to('cuda'), style_image.to('cuda'), input_img.to('cuda'))

  # plt.figure()
  # plt.title('Output Image')
  # plt.imshow(output.cpu().detach()[0].permute(1,2,0))

  # sphinx_gallery_thumbnail_number = 4
  # plt.ioff()
  # plt.show()

  # Save output image in ./StyleTransfer/
  torchvision.utils.save_image(output, "./StyleTransfer/Urban/" + urban_path[0].split('/')[-1])



  0%|          | 0/1156 [00:00<?, ?it/s]

Tried one
Building the style transfer model..


/usr/local/lib/python3.11/dist-packages/torch/utils/_device.py:106: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return func(*args, **kwargs)


Optimizing..
run [300]:
Style Loss : 0.911169 Content Loss: 3.888881



  0%|          | 1/1156 [00:20<6:31:16, 20.33s/it]

...

Tried one
Building the style transfer model..
Optimizing..
run [300]:
Style Loss : 0.905493 Content Loss: 4.214509



  1%|          | 8/1156 [02:35<6:13:18, 19.51s/it]

Tried one
Building the style transfer model..
Optimizing..
run [300]:
Style Loss : 0.375494 Content Loss: 2.610641



  1%|          | 9/1156 [02:55<6:13:48, 19.55s/it]

Tried one
Building the style transfer model..
Optimizing..


# Save styleTransfered images as zip


In [ ]:
# Save StyleTransfer directory as zip

import shutil

# Specify the directory to zip
directory_to_zip = "StyleTransfer"

# Specify the output zip file name
output_zip_file = "StyleTransfer.zip"

# Create a zip archive of the directory
shutil.make_archive(output_zip_file.replace('.zip', ''), 'zip', directory_to_zip)

# Verify the file is created
print(f"{output_zip_file} created successfully!")

# Optionally, download the zip file to your local system
from google.colab import files
files.download(output_zip_file)


In [ ]:
!cp StyleTransfer.zip /content/drive/MyDrive/LoveDA/